# Build a model of a protein family starting from a single sequence


1. Retrieve homologous (similar) sequences
2. Build a MSA from retrieved sequences
3. Caclulate a PSSM or HMM from the MSA
4. Evaluate the model against a ground truth dataset


### Implementation
For each step some alternative implementations that can have an effect on the model accuracy

**Step 1**
- Blast against a database (SwissProt, UniProt, RefSeq, NR, ReferenceProteomes, ...)

**Step 2**
- Build the MSA from Blast alignments 
- Build MSA realigning Blast hits (ClustalO, T-Coffee, Muscle, ...)
- Edit the MSA to remove poorly conserved columns
- Remove sequence redundancy, similar sequences, to improve model generalization
- Split the MSA, e.g. when there are multiple domains

**Step 3**
- Build a HMM using the HMMER command line 
- Build a PSSM using the Blast command line
- Skip this step and use the MSA in step 4 with HMMSEARCH

**Step 4**
- Define a ground truth dataset (Pfam, CATH, keywords, manual curation, ...)
- Calculate precision, recall, MCC, balanced accuracy





In [5]:
from Bio import SeqIO
from Bio.Blast import NCBIXML
import pandas as pd
from pathlib import Path

# Generate a MSA

Align P06621 with the [NCBI Blast webservice](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome)

Use to alternative databases
-SwissProt
-RefSeq selected

Set "Max target sequences" to 5000
When finish click on "Download" and select XML format

In [6]:

# Parse the XML output
data = []

# WARNING: if the code below crashes, remove "CREATE VIEW" lines manually from the XML file

# blast_input = 'data/P06621_sprot.xml'  # SwissProt database
blast_input = 'data/P06621_refseq_selected.xml'  # "RefSeq selected" database

with open(blast_input) as f:
    blast_records = NCBIXML.parse(f)

    # Iterate Psiblast rounds
    for blast_record in blast_records:
        
        # Iterate query alignments
        query_id = blast_record.query
        for i, alignment in enumerate(blast_record.alignments):
            subject_id = alignment.title
            
            for hsp in alignment.hsps:
                data.append((query_id,
                                subject_id,
                                blast_record.query_length,
                                hsp.query,
                                hsp.match,
                                hsp.sbjct,
                                hsp.query_start,
                                hsp.query_end,
                                hsp.sbjct_start,
                                hsp.sbjct_end,
                                hsp.identities,
                                hsp.positives,
                                hsp.gaps,
                                hsp.expect,
                                hsp.score))
                
df = pd.DataFrame(data, columns=["query_id", "subject_id", "query_len",
                              "query_seq", "match_seq", "subject_seq",
                              "query_start", "query_end", 
                              "subject_start", "subject_end", "identity", "positive", "gaps", "eval", "bit_score"])
df

,query_id,subject_id,query_len,query_seq,match_seq,subject_seq,query_start,query_end,subject_start,subject_end,identity,positive,gaps,eval,bit_score
0,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1545301071|ref|WP_126472623.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTAIAAVLATAF+AGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTAIAAVLATAFMAGTALAQKRDNVLFQAATDEQPAVIKT...,1,415,1,415,410,414,0,0.000000e+00,2129.0
1,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1543118946|ref|WP_126023768.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTA-LAQKRDNVLFQAATDEQPAVIK...,MRPSIHRTA+AA+LA AFVA A AQKRDNVLFQAATDEQPAVIK...,MRPSIHRTALAALLAAAFVAPAATWAQKRDNVLFQAATDEQPAVIK...,1,415,1,416,407,411,1,0.000000e+00,2097.0
2,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1797811451|ref|WP_159279459.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTA+AAVLATAFVAG A AQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTAVAAVLATAFVAGGAWAQKRDNVLFQAATDEQPAVIKT...,1,415,1,415,393,408,0,0.000000e+00,1992.0
3,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1480890214|ref|WP_119552493.1| M20/M25/M40 ...,415,MRPSIHRTAIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTA+AA+LATAF+A A AQKRDNVLFQAATDEQPAVIKT...,MRPSIHRTALAALLATAFLAPAAWAQKRDNVLFQAATDEQPAVIKT...,1,415,1,415,389,405,0,0.000000e+00,1978.0
4,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1011471139|ref|WP_062367304.1| M20/M25/M40 ...,415,MRPSIHRT-AIAAVLATAFVAGTALAQKRDNVLFQAATDEQPAVIK...,MRP RT A + AQKRDNVLFQAATD QPAV+K...,MRPFNQRTMLAALLATALLAPAAGWAQKRDNVLFQAATDAQPAVLK...,1,415,1,416,374,393,1,0.000000e+00,1938.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1510926788|ref|WP_122926114.1| ArgE/DapE fa...,415,EQPAVIKTLEKLVNIETGT----GDAEGIAAAGNFLEAELKNLGFT...,E+ +I ++ L+ I++ D G +L+A+L +G ...,ERDELIGLVQDLIRIDSVNPYLDADGPGEREMAAYLQAKLVEMGLE...,39,261,6,223,70,117,19,2.135720e-12,190.0
4997,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1460625799|ref|WP_116468663.1| hydrolase [S...,415,LVVGDNIVGKIKGRGGKNLLLMSHMDTVY-LKGILAKAPFRVEGDK...,L G+++ ++ +L HMDTV+ + +R +G ...,LAHGEHLHLVVRPEAPIQMLFTGHMDTVFGADHVFQHGQWRTDG-T...,89,411,78,396,93,150,18,2.144680e-12,190.0
4998,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1718634206|ref|WP_145030192.1| M20/M25/M40 ...,415,LFQAATDEQPAVIKTLEKLVNIETGTGDAEGIAAAGNFLEAELKNL...,+ +A T E A I + +L++I +G+ +G+A +FL +LK ...,MARAKTSENKAAIDLVLELLSISGKSGEEKGVA---DFLVKKLKGA...,32,237,1,210,73,106,24,2.197150e-12,190.0
4999,sp|P06621|CBPG_PSES6 Carboxypeptidase G2 OS=Ps...,gi|1393435437|ref|WP_109798908.1| M20/M25/M40 ...,415,IAAVLATAFVAGTALAQKRDN----VLFQAATDEQPAVIKTLEKLV...,IA+ LA + A A+A+ ++ + EQ +K LE LV...,IASALALSCSAVQAMAKGPESGPEARMIATIDAEQTRTLKFLETLV...,10,339,7,361,96,151,27,2.322320e-12,190.0


In [7]:
# Extract the sequence of subject proteins (hits) aligned with the query to build the MSA

input_path = Path(blast_input)
msa = []

with open("{}/{}_blast_msa.fasta".format(input_path.parent, input_path.stem), "w") as fout:
    for index, row in df.iterrows():
        mapped_seq = ["-"] * blast_record.query_length  # Empty list of length = query_length
        c = 0
        if row["eval"] < 0.001:
        #     print(row)
            for l_q, l_s in zip(row['query_seq'], row['subject_seq']):
                if l_q != " " and l_q != '-':
                    mapped_seq[row["query_start"] + c -1] = l_s if l_s != " " else "-"
                    c += 1
            fout.write(">{}\n{}\n".format(row["subject_id"], "".join(mapped_seq)))



# Generate a better MSA

Generate a new, possibly better, MSA starting from the full sequences of hits retrieved by Blast using the Clustal Omega method

1. Extract the protein IDs from previous files

```
    # SwissProt IDs
    awk -F"|" '/>/ {print $2}' data/P06621_sprot_blast_msa.fasta > tmp
    
    # RefSeq IDs (ex. WP_094475116.1)
    awk -F"|" '/>/ {print $4}' data/P06621_refseq_selected_blast_msa.fasta > tmp2
   
``` 

2. Download the sequences from UniProt using the **Retrieve/ID mapping** web tool
    * For SwissProt IDs select "From --> UniProtKB, To UniProtKB. Download into **P06621_sprot_blast_mapped.fasta** 
    * For RefSeq selected "From --> RefSeq Protein, To UniProtKB. Select "View UniParc results" to increase the number of retrieved sequences. Download into **P06621_refseq_selected_blast_mapped.fasta**


3. Use [ClustalO](https://www.ebi.ac.uk/Tools/msa/clustalo/) to generate new MSAs. Select the *Fasta* output format and save into
    * **P06621_sprot_blast_clustal.fasta**
    * **P06621_refseq_selected_blast_clustal.fasta**


4. Compare the Blast MSA and the ClustalO MSA in JalView

```
    java -jar jalview-all-2.11.1.4-j1.8.jar
```

# Build PSSMs

To build PSSM from a MSA file it is necesary to use the command line version of Blast

#### Install Blast (~250 Mb)
```
wget ftp://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.12.0+-x64-linux.tar.gz
tar -xzf ncbi-blast-2.12.0+-x64-linux.tar.gz

```

#### Create a PSSM from a Fasta MSA 

Before running the command remove *gi|1774282670|ref|WP_153585536.1|* from the **data/P06621_refseq_selected_blast_msa.fasta** file, for some reasons it breaks the PSSM build

The content of the file in the -subject option is irrelevant, just use a valid fasta file
```
ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_sprot_blast_msa.fasta -out_pssm data/P06621_sprot_blast_msa.pssm -out_ascii_pssm data/P06621_sprot_blast_msa.pssm_ascii

ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_refseq_selected_blast_msa.fasta -out_pssm data/P06621_refseq_selected_blast_msa.pssm -out_ascii_pssm data/P06621_refseq_selected_blast_msa.pssm_ascii
```

# Build HMM

Install HMMER (~18 Mb)
```
wget http://eddylab.org/software/hmmer/hmmer.tar.gz
tar -xzf hmmer.tar.gz
./configure
make
```

Generate HMMs from MSAs
```
hmmer-3.3.2/src/hmmbuild data/P06621_sprot_blast_msa.hmm data/P06621_sprot_blast_msa.fasta

hmmer-3.3.2/src/hmmbuild data/P06621_refseq_selected_blast_msa.hmm data/P06621_refseq_selected_blast_msa.fasta
```

# Build HMM and PSSM from ClustalO MSAs

Generate PSSMs
```
ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_sprot_blast_clustal.fasta -out_pssm data/P06621_sprot_blast_clustal.pssm -out_ascii_pssm data/P06621_sprot_blast_clustal.pssm_ascii

ncbi-blast-2.12.0+/bin/psiblast -subject data/P06621.fasta -in_msa data/P06621_refseq_selected_blast_clustal.fasta -out_pssm data/P06621_refseq_selected_blast_clustal.pssm -out_ascii_pssm data/P06621_refseq_selected_blast_clustal.pssm_ascii
```

Generate HMMs
```
hmmer-3.3.2/src/hmmbuild data/P06621_sprot_blast_clustal.hmm data/P06621_sprot_blast_clustal.fasta

hmmer-3.3.2/src/hmmbuild data/P06621_refseq_selected_blast_clustal.hmm data/P06621_refseq_selected_blast_clustal.fasta
```

# Evaluation

Compare performance of your models in comparison with Pfam **Peptidase_M20.hmm**. 

1. Evaluate the number of hits retrieved by your models and those retrived by the Pfam domain aginst the SwissProt database.

    * **PSSMs** You can use the web service. When you select Psi-Blast it is possible to provide a PSSM file as input

    * **HMMs** You can use HMMSEARCH from HMMER web site


2. Evaluate the overlap between the hits retrieved by your models and the hits retrived by the Pfam model

    * You need to download sequence IDs of the matched sequences 


3. Evaluate False Positive matches (if any) and decide if they are good family candidates missed by Pfam but not by your model

    * You need to look at alternative information to judge. Statistical parameters, curated annotations, etc...
    


In [25]:
ground_truth = pd.read_csv('data/Peptidase_M20_sprot_hmmsearch.tsv', sep='\t')
ground_truth


0       DAPE_CHESB
1       DAPE_RHILO
2       DAPE_BRUA1
3       DAPE_BRUA2
4       DAPE_BRUAB
           ...    
1016    LAP4_ARTBC
1017    PFF1_PYRTT
1018    BSAP_BACSU
1019    PFF1_KOMPG
1020    LAP4_ARTOC
Name: Target Accession, Length: 1021, dtype: object

In [4]:
# hmmsearch from the Blast MSA (HMM)
hmmsearch_blast_msa = pd.read_csv('data/P06621_refseq_selected_blast_msa_hmmsearch.tsv', sep='\t')
hmmsearch_blast_msa

,Target Name,Target Accession,Target Length,Query Name,Query Accession,Query Length,E-value,Score,Bias,Domain Index,...,Query Ali. Start,Query Ali. End,Target Ali. Start,Target Ali. End,Target Env. Start,Target Env. End,Acc,Description,Mapped PDB(s),Number of Identical Sequences
0,CBPG_PSES6,CBPG_PSES6,415,P06621_refseq_selected_blast_msa,-,368,5.900000e-144,484.4,1.6,1,...,2,368,44,411,43,411,1.00,Carboxypeptidase G2,7m6u_D,0
1,YQJE_BACSU,YQJE_BACSU,371,P06621_refseq_selected_blast_msa,-,368,4.900000e-68,234.6,9.3,1,...,2,366,8,367,7,369,0.91,Uncharacterized protein YqjE,NaN,0
2,DAPE2_SHELP,DAPE2_SHELP,377,P06621_refseq_selected_blast_msa,-,368,7.700000e-46,161.5,0.0,1,...,9,366,8,373,2,375,0.87,Succinyl-diaminopimelate desuccinylase 2,NaN,0
3,DAPE_DECAR,DAPE_DECAR,377,P06621_refseq_selected_blast_msa,-,368,9.000000e-46,161.3,0.0,1,...,2,367,6,373,5,374,0.87,Succinyl-diaminopimelate desuccinylase,NaN,0
4,DAPE_ALKEH,DAPE_ALKEH,375,P06621_refseq_selected_blast_msa,-,368,1.400000e-45,160.7,0.0,1,...,10,364,13,369,4,373,0.83,Succinyl-diaminopimelate desuccinylase,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,LAP1_PARBA,LAP1_PARBA,389,P06621_refseq_selected_blast_msa,-,368,7.400000e-03,20.1,0.1,1,...,7,148,130,261,126,270,0.79,Leucine aminopeptidase 1,NaN,0
1046,LAP1_SCLS1,LAP1_SCLS1,387,P06621_refseq_selected_blast_msa,-,368,7.500000e-03,20.1,0.1,1,...,35,149,154,260,133,282,0.79,Leucine aminopeptidase 1,NaN,0
1047,PFF1_PARBD,PFF1_PARBD,992,P06621_refseq_selected_blast_msa,-,368,8.100000e-03,20.0,0.0,1,...,90,165,177,253,161,262,0.87,Vacuolar membrane protease,NaN,0
1048,LAP1_PODAN,LAP1_PODAN,395,P06621_refseq_selected_blast_msa,-,368,8.200000e-03,20.0,0.1,1,...,45,149,163,261,138,274,0.77,Leucine aminopeptidase 1,NaN,0


In [19]:
# PSI-BLAST search with the Blast MSA (PSSM, increase 20k limit)
# Download --> Hit Table (CSV)
psiblast_blast_pssm = pd.read_csv('data/P06621_refseq_selected_blast_msa_psiblast.csv', header=None)

# Map ids with the UniProt id mapping service 
# Alternatively download a mapping file from here (huge file) 
# https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/proteomics_mapping/

psiblast_blast_pssm[psiblast_blast_pssm[10] < 0.01][1].to_csv(
    'data/P06621_refseq_selected_blast_msa_psiblast.ids', index=False, header=False)

In [22]:
psiblast_blast_pssm = pd.read_csv('data/P06621_refseq_selected_blast_msa_psiblast_mapped.tsv', sep='\t')
psiblast_blast_pssm

,yourlist:M202111216320BA52A5CE8FCD097CB85A53697A35307ECBV,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,P06621,P06621,CBPG_PSES6,reviewed,Carboxypeptidase G2 (CPDG2) (EC 3.4.17.11) (Fo...,cpg2,Pseudomonas sp. (strain RS-16),415
1,Q5FPX5,Q5FPX5,DAPE_GLUOX,reviewed,Succinyl-diaminopimelate desuccinylase (SDAP d...,dapE GOX1832,Gluconobacter oxydans (strain 621H) (Gluconoba...,381
2,A9ILD7,A9ILD7,DAPE_BART1,reviewed,Succinyl-diaminopimelate desuccinylase (SDAP d...,dapE BT_0051,Bartonella tribocorum (strain CIP 105476 / IBS...,390
3,A5ESQ3,A5ESQ3,DAPE_BRASB,reviewed,Succinyl-diaminopimelate desuccinylase (SDAP d...,dapE BBta_7333,Bradyrhizobium sp. (strain BTAi1 / ATCC BAA-1182),384
4,Q59284,Q59284,DAPE_CORGL,reviewed,Succinyl-diaminopimelate desuccinylase (SDAP d...,dapE Cgl1109 cg1260,Corynebacterium glutamicum (strain ATCC 13032 ...,369
...,...,...,...,...,...,...,...,...
169,Q0SWT9,Q0SWT9,PEPT_CLOPS,reviewed,Peptidase T (EC 3.4.11.4) (Aminotripeptidase) ...,pepT CPR_0029,Clostridium perfringens (strain SM101 / Type A),406
170,A1JL17,A1JL17,DAPE_YERE8,reviewed,Succinyl-diaminopimelate desuccinylase (SDAP d...,dapE YE1147,Yersinia enterocolitica serotype O:8 / biotype...,375
171,O50173,O50173,IAAH_ENTAG,reviewed,Indole-3-acetyl-aspartic acid hydrolase (EC 3....,iaaH,Enterobacter agglomerans (Erwinia herbicola) (...,436
172,Q4QP83,Q4QP83,DAPE_HAEI8,reviewed,Succinyl-diaminopimelate desuccinylase (SDAP d...,dapE NTHI0182,Haemophilus influenzae (strain 86-028NP),377


In [27]:
gt = set(ground_truth[ground_truth['E-value']<0.01]['Target Accession'])
hmm = set(hmmsearch_blast_msa[hmmsearch_blast_msa['E-value']<0.01]['Target Accession'])
pssm = set(psiblast_blast_pssm['Entry name'])

for name, data in zip(['hmm', 'pssm'], [hmm, pssm]):
    print(name, 'FN', len(gt - data))
    print(name, 'TP', len(gt.intersection(data)))
    print(name, 'FP', len(data - gt))

hmm FN 49
hmm TP 970
hmm FP 5
pssm FN 846
pssm TP 173
pssm FP 1


# Improve your model

Manually edit the MSA in order to achieve better precision and/or recall

- Identify conserved positions (ex. evaluate column entropy)
- Remove redundancy (ex. use JalView or CD-HIT)


#### Shannon entropy

![Entropy](figures/entropy.png)
scipy.stats.entropy [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html)

In [3]:
from Bio import AlignIO
import scipy.stats
from collections import Counter
import numpy as np

In [5]:

print(scipy.stats.entropy([1/20 for i in range(20)], base=20))


aa = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

seqs = []  # [[...], ...]
# with open("data/P06621_refseq_selected_blast_msa.fasta") as f:
with open("data/P06621_refseq_selected_blast_clustal.fasta") as f:
    for record in AlignIO.read(f, "fasta"):
        seqs.append(list(record.seq))  # store sequence as a list of characters


seqs = np.array(seqs, dtype="str")
for i, column in enumerate(seqs.T):
    
    # count AA in column
    count = Counter(column)
    count.pop('-')
    count_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)
    
    # count non gap AA
    non_gap = np.count_nonzero(column != "-")
    
    occupancy = non_gap / column.size
    
    # AA probability in column (gap excluded)
#     probabilities = [count.get(k, 0.0) / column.size for k in aa]  
    probabilities = [count.get(k, 0.0) / non_gap for k in aa]  

    # Zero entropy = complete conservation
    entropy = scipy.stats.entropy(probabilities, base=20)  

    print("{} {:>5.2f} {:>5.2f} {}".format(i, 
                                         occupancy, 
                                         entropy,
                                         count_sorted))

0.9999999999999999
0  0.00   inf [('M', 2)]
1  0.00   inf [('S', 1), ('T', 1)]
2  0.00   inf [('D', 2), ('M', 1)]
3  0.00   inf [('T', 1)]
4  0.00   inf [('P', 2), ('S', 1)]
5  0.00   inf [('H', 1), ('D', 1), ('S', 1)]
6  0.01   inf [('M', 17), ('E', 1), ('T', 1), ('P', 1)]
7  0.01   inf [('S', 7), ('N', 6), ('T', 4), ('K', 1), ('H', 1), ('Q', 1)]
8  0.00   inf [('D', 1)]
9  0.00   inf [('P', 1)]
10  0.01   inf [('D', 16), ('S', 2), ('M', 1), ('A', 1), ('E', 1)]
11  0.01   inf [('T', 9), ('P', 4), ('Q', 2), ('I', 2), ('A', 2), ('G', 1), ('E', 1)]
12  0.01   inf [('S', 5), ('T', 5), ('M', 3), ('Q', 3), ('H', 3), ('V', 1), ('F', 1), ('A', 1), ('E', 1), ('R', 1)]
13  0.01   inf [('P', 12), ('M', 10), ('Q', 5), ('S', 4), ('E', 1), ('R', 1), ('A', 1)]
14  0.01   inf [('E', 7), ('S', 6), ('Q', 5), ('K', 4), ('T', 4), ('D', 2), ('V', 2), ('P', 1), ('A', 1), ('N', 1)]
15  0.01   inf [('R', 13), ('G', 7), ('A', 3), ('T', 3), ('P', 2), ('K', 2), ('H', 1), ('I', 1), ('D', 1), ('S', 1)]
16  0.01  

282  0.01   inf [('D', 21), ('A', 5), ('S', 3), ('H', 2), ('V', 2), ('P', 2), ('R', 1), ('G', 1)]
283  0.01   inf [('P', 28), ('A', 5), ('E', 3), ('T', 2), ('S', 2), ('Q', 1)]
284  0.01   inf [('A', 20), ('E', 6), ('T', 4), ('G', 4), ('V', 3), ('P', 1), ('H', 1), ('N', 1)]
285  0.01   inf [('A', 17), ('V', 8), ('T', 6), ('D', 4), ('G', 3), ('E', 2), ('R', 1)]
286  0.01   inf [('R', 33), ('A', 3), ('T', 2), ('Q', 2), ('S', 1)]
287  0.01   inf [('D', 14), ('E', 7), ('R', 6), ('S', 3), ('T', 3), ('A', 3), ('N', 2), ('P', 1), ('G', 1)]
288  0.01   inf [('A', 29), ('T', 4), ('D', 2), ('S', 2), ('G', 2)]
289  0.01   inf [('A', 33), ('E', 2), ('T', 1), ('G', 1)]
290  0.01   inf [('F', 17), ('T', 12), ('C', 5), ('P', 2)]
291  0.01   inf [('D', 30), ('A', 6), ('C', 1)]
292  0.01   inf [('R', 35), ('G', 1), ('A', 1)]
293  0.01   inf [('L', 34), ('S', 1), ('T', 1), ('A', 1)]
294  0.01   inf [('T', 31), ('V', 2), ('I', 2), ('A', 1), ('E', 1), ('Q', 1), ('S', 1)]
295  0.01   inf [('A', 34), ('P', 2

585  0.00   inf [('G', 7), ('A', 1), ('S', 1)]
586  0.93  0.54 [('L', 2335), ('I', 283), ('A', 252), ('M', 180), ('T', 96), ('H', 78), ('V', 68), ('D', 64), ('F', 53), ('Q', 35), ('E', 32), ('S', 22), ('R', 17), ('Y', 12), ('C', 10), ('G', 7), ('N', 7), ('P', 5), ('W', 4), ('K', 3)]
587  0.15   inf [('P', 269), ('G', 92), ('A', 65), ('I', 36), ('E', 34), ('L', 19), ('D', 17), ('S', 17), ('T', 5), ('N', 5), ('C', 2), ('Q', 2), ('K', 1)]
588  0.98   inf [('G', 3280), ('D', 146), ('P', 109), ('A', 51), ('N', 43), ('S', 26), ('Q', 24), ('E', 20), ('R', 18), ('L', 17), ('H', 16), ('K', 8), ('F', 6), ('C', 5), ('V', 5), ('I', 4), ('T', 3), ('M', 3)]
589  0.00   inf [('A', 4)]
590  0.89  0.32 [('A', 899), ('F', 780), ('G', 358), ('L', 245), ('V', 217), ('I', 216), ('W', 149), ('R', 97), ('M', 77), ('E', 66), ('T', 62), ('C', 59), ('P', 56), ('Y', 54), ('S', 48), ('Q', 20), ('K', 14), ('D', 11), ('H', 9), ('N', 1)]
591  0.00   inf [('N', 2), ('D', 2), ('S', 1), ('Q', 1), ('F', 1)]
592  0.56  0

815  0.00   inf [('A', 2), ('R', 1)]
816  0.00   inf [('V', 1), ('G', 1), ('A', 1)]
817  0.00   inf [('P', 2), ('A', 1)]
818  0.00   inf [('P', 1), ('R', 1)]
819  0.00   inf [('T', 1), ('P', 1)]
820  0.00   inf [('D', 1), ('E', 1)]
821  0.00   inf [('G', 3)]
822  0.00   inf [('L', 1), ('A', 1)]
823  0.00   inf [('D', 1), ('A', 1), ('E', 1)]
824  0.00   inf [('H', 3), ('T', 2), ('D', 1), ('G', 1), ('R', 1), ('Q', 1), ('N', 1), ('E', 1)]
825  0.00   inf [('S', 5), ('R', 2), ('D', 1), ('G', 1), ('Q', 1), ('N', 1)]
826  0.00   inf [('D', 4), ('L', 1), ('A', 1), ('E', 1), ('K', 1)]
827  0.00   inf [('I', 5), ('G', 4), ('M', 1), ('Q', 1), ('N', 1)]
828  0.06   inf [('G', 151), ('N', 18), ('A', 11), ('F', 10), ('Q', 8), ('P', 8), ('D', 7), ('Y', 5), ('C', 5), ('S', 4), ('T', 3), ('H', 3), ('L', 3), ('K', 2), ('E', 1), ('V', 1), ('R', 1)]
829  0.55   inf [('G', 920), ('P', 285), ('N', 202), ('D', 164), ('K', 97), ('E', 86), ('A', 73), ('Q', 67), ('V', 67), ('H', 44), ('S', 41), ('R', 25), ('L'

1273  0.00   inf [('I', 1)]
1274  0.06   inf [('A', 98), ('V', 29), ('T', 27), ('S', 24), ('L', 17), ('E', 16), ('R', 8), ('G', 7), ('I', 4), ('C', 3), ('F', 2), ('Q', 2), ('K', 2), ('H', 2), ('W', 1), ('N', 1), ('Y', 1), ('M', 1)]
1275  0.36  0.30 [('N', 204), ('T', 197), ('A', 190), ('K', 172), ('E', 123), ('R', 121), ('Q', 84), ('D', 73), ('S', 56), ('H', 53), ('G', 34), ('V', 20), ('C', 17), ('L', 10), ('F', 8), ('Y', 5), ('I', 5), ('P', 4), ('M', 3), ('W', 1)]
1276  0.44   inf [('P', 316), ('K', 223), ('S', 208), ('H', 148), ('E', 137), ('T', 135), ('N', 131), ('A', 98), ('Q', 89), ('R', 73), ('G', 39), ('D', 32), ('V', 30), ('I', 24), ('L', 8), ('C', 8), ('M', 3), ('Y', 2), ('F', 2)]
1277  0.00   inf [('A', 6)]
1278  0.00   inf [('A', 4), ('E', 1)]
1279  0.44  0.22 [('L', 561), ('F', 185), ('Y', 140), ('E', 130), ('K', 103), ('A', 89), ('D', 84), ('S', 84), ('T', 65), ('V', 46), ('R', 46), ('H', 43), ('G', 29), ('Q', 27), ('W', 21), ('N', 16), ('I', 11), ('P', 11), ('M', 8), ('C'

1623  0.00   inf [('S', 1)]
1624  0.00   inf [('L', 1)]
1625  0.00   inf [('L', 1)]
1626  0.00   inf [('A', 1)]
1627  0.00   inf [('G', 1)]
1628  0.00   inf [('V', 1)]
1629  0.00   inf [('L', 1)]
1630  0.00   inf [('P', 1)]
1631  0.00   inf [('G', 1)]
1632  0.00   inf [('N', 1)]
1633  0.00   inf [('A', 1)]
1634  0.00   inf [('D', 1)]
1635  0.00   inf [('Q', 1)]
1636  0.00   inf [('G', 1)]
1637  0.00   inf [('V', 1)]
1638  0.00   inf [('G', 1)]
1639  0.00   inf [('F', 1)]
1640  0.00   inf [('A', 1)]
1641  0.00   inf [('L', 1)]
1642  0.00   inf [('K', 1)]
1643  0.00   inf [('Q', 1)]
1644  0.00   inf [('H', 1)]
1645  0.00   inf [('Q', 1)]
1646  0.00   inf [('R', 1)]
1647  0.00   inf [('A', 1)]
1648  0.00   inf [('W', 1)]
1649  0.00   inf [('A', 1)]
1650  0.00   inf [('L', 1)]
1651  0.00   inf [('A', 1)]
1652  0.00   inf [('R', 1)]
1653  0.00   inf [('D', 1)]
1654  0.00   inf [('I', 1)]
1655  0.00   inf [('E', 1)]
1656  0.00   inf [('A', 1)]
1657  0.00   inf [('I', 1)]
1658  0.00   inf [('